In [1]:
import os
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
from google.colab.patches import cv_imshow

In [2]:
(train_X, train_Y), (test_X, test_Y) = tf.keras.datasets.cifar10.load_data()

In [3]:
np.unique(train_Y)

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9], dtype=uint8)

0: airplane
1: automobile
2: bird
3: cat
4: deer
5: dog
6: forg
7: horse
8: ship
9: truck  
=> 0,1,8

In [4]:
specific_idx = np.where((train_Y == 0) | (train_Y == 1) | (train_Y == 8))
np.shape(specific_idx[0])

(15000,)

In [5]:
new_train_X = train_X[specific_idx[0]]
new_train_Y = train_Y[specific_idx[0]]

In [6]:
print(np.shape(train_X))
print(np.shape(new_train_X))

(50000, 32, 32, 3)
(15000, 32, 32, 3)


In [7]:
specific_idx2 = np.where((test_Y == 0) | (test_Y == 1) | (test_Y == 8))
np.shape(specific_idx2[0])

(3000,)

In [8]:
new_test_X = test_X[specific_idx2[0]]
new_test_Y = test_Y[specific_idx2[0]]

In [9]:
print(np.shape(test_X))
print(np.shape(new_test_X))

(10000, 32, 32, 3)
(3000, 32, 32, 3)


In [10]:
new_train_Y[np.where(new_train_Y ==8)] = 2
new_test_Y[np.where(new_test_Y ==8)] = 2

In [11]:
print(np.unique(new_train_Y))
print(np.unique(new_test_Y))

[0 1 2]
[0 1 2]


# Data Preprocessing

In [12]:
new_train_X = tf.keras.applications.vgg16.preprocess_input(new_train_X)
new_test_X = tf.keras.applications.vgg16.preprocess_input(new_test_X)

In [13]:
base_model = tf.keras.applications.VGG16(input_shape=[32,32,3],
                                         include_top=False, weights='imagenet')
x= base_model.output
x= tf.keras.layers.Flatten()(x)
x= tf.keras.layers.Dense(64, activation='relu')(x)
predictions= tf.keras.layers.Dense(3, activation='softmax')(x)
model =tf.keras.Model(inputs=base_model.input, outputs=predictions)

In [14]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [15]:
for layer in model.layers[:19]:
  layer.trainable = False
for layer in model.layers[19:]:
  layer.trainable = True
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [16]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss= 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [17]:
history = model.fit(new_train_X, new_train_Y, batch_size=100, epochs=10)

Epoch 1/10
150/150 [==============================] - 14s 17ms/step - loss: 1.5379 - accuracy: 0.7477
Epoch 2/10
150/150 [==============================] - 3s 17ms/step - loss: 0.5031 - accuracy: 0.8289
Epoch 3/10
150/150 [==============================] - 3s 17ms/step - loss: 0.3476 - accuracy: 0.8724
Epoch 4/10
150/150 [==============================] - 3s 17ms/step - loss: 0.2733 - accuracy: 0.8977
Epoch 5/10
150/150 [==============================] - 3s 17ms/step - loss: 0.2277 - accuracy: 0.9117
Epoch 6/10
150/150 [==============================] - 3s 18ms/step - loss: 0.1918 - accuracy: 0.9271
Epoch 7/10
150/150 [==============================] - 3s 17ms/step - loss: 0.1636 - accuracy: 0.9409
Epoch 8/10
150/150 [==============================] - 3s 17ms/step - loss: 0.1397 - accuracy: 0.9511
Epoch 9/10
150/150 [==============================] - 3s 17ms/step - loss: 0.1195 - accuracy: 0.9577
Epoch 10/10
150/150 [==============================] - 3s 17ms/step - loss: 0.1008 - accur

In [21]:
model.evaluate(new_test_X, new_test_Y, verbose = False)

[0.6154810190200806, 0.8483333587646484]

In [15]:
for layer in model.layers:
  layer.trainable = True
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 32, 32, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 32, 32, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 16, 16, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 16, 16, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 16, 16, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 8, 8, 128)         0     

In [16]:
model.compile(optimizer = tf.keras.optimizers.Adam(),
              loss= 'sparse_categorical_crossentropy',
              metrics = ['accuracy'])

In [17]:
history = model.fit(new_train_X, new_train_Y, batch_size=100, epochs=10)

Epoch 1/10
150/150 [==============================] - 12s 47ms/step - loss: 1.6839 - accuracy: 0.4834
Epoch 2/10
150/150 [==============================] - 7s 47ms/step - loss: 0.6547 - accuracy: 0.7061
Epoch 3/10
150/150 [==============================] - 7s 48ms/step - loss: 0.4495 - accuracy: 0.8196
Epoch 4/10
150/150 [==============================] - 7s 48ms/step - loss: 0.3423 - accuracy: 0.8695
Epoch 5/10
150/150 [==============================] - 7s 49ms/step - loss: 0.2279 - accuracy: 0.9162
Epoch 6/10
150/150 [==============================] - 7s 49ms/step - loss: 0.1730 - accuracy: 0.9363
Epoch 7/10
150/150 [==============================] - 7s 49ms/step - loss: 0.1422 - accuracy: 0.9491
Epoch 8/10
150/150 [==============================] - 7s 49ms/step - loss: 0.1040 - accuracy: 0.9615
Epoch 9/10
150/150 [==============================] - 7s 49ms/step - loss: 0.1145 - accuracy: 0.9589
Epoch 10/10
150/150 [==============================] - 7s 50ms/step - loss: 0.0835 - accur

In [21]:
model.evaluate(new_test_X, new_test_Y, verbose = False)

[0.19454748928546906, 0.9359999895095825]